<a href="https://colab.research.google.com/github/t-zakharenko/ML-course-May-2025/blob/main/HW_2_4_kNN_%D0%9A%D1%80%D0%BE%D1%81%D0%B2%D0%B0%D0%BB%D1%96%D0%B4%D0%B0%D1%86%D1%96%D1%8F_%D1%96_%D1%82%D1%8E%D0%BD%D0%B8%D0%BD%D0%B3_%D0%B3%D1%96%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D1%96%D0%B2_TZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

!ls

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive			 __pycache__  sample_submission.csv  train.csv
process_bank_churn_2.py  sample_data  test.csv
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# from sklearn.tree import DecisionTreeClassifier, plot_tree
# from sklearn.metrics import roc_auc_score
# import matplotlib.pyplot as plt


import pandas as pd



In [7]:
from process_bank_churn_2 import preprocess_data, preprocess_new_data

In [8]:
# Завантажуємо дані
raw_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [9]:
# Попередня обробка
X_train, y_train, X_val, y_val, input_cols, scaler, encoder = preprocess_data(raw_df)

/content/process_bank_churn_2.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'Gender': {'Female': 0, 'Male': 1}})


1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [10]:
!pip install mlxtend

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlxtend.plotting import plot_decision_regions

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [12]:
Xy_train = X_train.copy()
Xy_train['Exited'] = y_train

Xy_train.corr().Exited.to_frame().style.background_gradient()

,Exited
Gender,-0.174172
Age,0.473893
Tenure,-0.010693
Balance,0.151222
NumOfProducts,-0.286473
HasCrCard,-0.018979
IsActiveMember,-0.187777
EstimatedSalary,0.011588
Geography_France,-0.132076
Geography_Germany,0.242515


In [13]:
# Навчіть на цих даних класифікатор kNN з параметрами за замовченням
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [14]:
train_probs = knn.predict_proba(X_train)[:, 1]
val_probs = knn.predict_proba(X_val)[:, 1]

In [15]:
# виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах

from sklearn.metrics import roc_auc_score

train_auc = roc_auc_score(y_train, train_probs)
val_auc = roc_auc_score(y_val, val_probs)

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")


Train AUROC: 0.9589
Validation AUROC: 0.8779


Висока точність на тренувальному наборі, але помітно менша - на валідаційному.
Це сідчить про High Variance (висока варіативність), модель перенавчилася на тренувальних даних і погано узагальнє нові дані. Треба працювати далі з підбором гіперпараметрів з метою зменшення варіативності.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Поставте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [16]:
# Створюємо модель
knn = KNeighborsClassifier()

# Визначаємо параметри для перебору
params_knn = {'n_neighbors': np.arange(1, 25)}

# Налаштовуємо GridSearchCV
knn_grid_search = GridSearchCV(knn, params_knn, cv=5)

# Запускаємо перебір
knn_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [17]:
# Виводимо найкраще значення n_neighbors
print("Best n_neighbors:", knn_grid_search.best_params_['n_neighbors'])

# Зберігаємо найкращу модель
knn_best = knn_grid_search.best_estimator_

Best n_neighbors: 5


In [18]:
# Оцінюємо AUROC на тренувальній і валідаційній вибірці
from sklearn.metrics import roc_auc_score

train_probs_best = knn_best.predict_proba(X_train)[:, 1]
val_probs_best = knn_best.predict_proba(X_val)[:, 1]

train_auc_best = roc_auc_score(y_train, train_probs_best)
val_auc_best = roc_auc_score(y_val, val_probs_best)

print(f"Train AUROC (best k): {train_auc_best:.4f}")
print(f"Validation AUROC (best k): {val_auc_best:.4f}")


Train AUROC (best k): 0.9589
Validation AUROC (best k): 0.8779


зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

Якість моделі не змінилася (не покращилася), це викликано тим, що знайдений найкращий параметр збігається з тим, що стоїть в моделі за замовченням.

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np
import time

In [20]:
# Сітка параметрів
params_dt_grid = {
    'max_depth': np.arange(1, 21, 2),          # від 1 до 20 з кроком 2
    'max_leaf_nodes': np.arange(2, 11, 1)      # від 2 до 10 з кроком 1
}
params_dt_grid

{'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
 'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])}

In [21]:
# Ініціалізація моделі з фіксованим random_state = 42
dt_model = DecisionTreeClassifier(random_state=42)

In [22]:
# Кросвалідація
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [23]:
# Налаштування GridSearchCV
grid_search_dt = GridSearchCV(
    estimator=dt_model,
    param_grid=params_dt_grid,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

In [24]:
# Замір часу
start_time = time.time()
grid_search_dt.fit(X_train, y_train)
end_time = time.time()

print(f"Час пошуку: {end_time - start_time:.2f} секунд")

Fitting 3 folds for each of 90 candidates, totalling 270 fits
Час пошуку: 6.25 секунд


In [25]:
# виведіть найкращі значення параметра
# збережіть в окрему змінну dt_best найкращу модель, знайдену з GridSearchCV
# оцініть якість передбачень dt_best на тренувальній і валідаційній вибірці з допомогою AUROC.
# зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?



# Виведення найкращих параметрів
print("Найкращі параметри:", grid_search_dt.best_params_)

# Збереження найкращої моделі
dt_best = grid_search_dt.best_estimator_

# Оцінка AUROC
train_probs_dt_best = dt_best.predict_proba(X_train)[:, 1]
val_probs_dt_best = dt_best.predict_proba(X_val)[:, 1]

train_auc_dt_best = roc_auc_score(y_train, train_probs_dt_best)
val_auc_dt_best = roc_auc_score(y_val, val_probs_dt_best)

print(f"Train AUROC (GridSearch DT): {train_auc_dt_best:.4f}")
print(f"Validation AUROC (GridSearch DT): {val_auc_dt_best:.4f}")

Найкращі параметри: {'max_depth': np.int64(5), 'max_leaf_nodes': np.int64(10)}
Train AUROC (GridSearch DT): 0.9010
Validation AUROC (GridSearch DT): 0.9018


Validation AUROC майже не відрізняється від Train AUROC (навіть трохи більше), і це свідчить про те, що модель добре узагальнює, немає overfitting.

В порівнянні з моделлю kNN ця модель (дерево рішень) демонструє значно кращий результат.


4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np
import time

In [27]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [28]:
# Ініціалізація моделі
dt = DecisionTreeClassifier(random_state=42)

In [29]:
# Крос-валідація з фіксацією seed
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [30]:
# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=params_dt,
    n_iter=40,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

In [31]:
# Вимірюємо час
start_time = time.time()
random_search.fit(X_train, y_train)
end_time = time.time()
print(f"Час пошуку: {end_time - start_time:.2f} секунд")

Fitting 3 folds for each of 40 candidates, totalling 120 fits
Час пошуку: 1.35 секунд


In [32]:
# Найкращі параметри
print("Найкращі параметри:", random_search.best_params_)

# Збереження моделі
dt_random_search_best = random_search.best_estimator_

# Оцінка AUROC
train_probs_dt = dt_random_search_best.predict_proba(X_train)[:, 1]
val_probs_dt = dt_random_search_best.predict_proba(X_val)[:, 1]

train_auc_dt = roc_auc_score(y_train, train_probs_dt)
val_auc_dt = roc_auc_score(y_val, val_probs_dt)

print(f"Train AUROC (Decision Tree): {train_auc_dt:.4f}")
print(f"Validation AUROC (Decision Tree): {val_auc_dt:.4f}")

Найкращі параметри: {'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': np.int64(14), 'max_features': None, 'max_depth': np.int64(16), 'criterion': 'entropy'}
Train AUROC (Decision Tree): 0.9166
Validation AUROC (Decision Tree): 0.9176


In [33]:
submission_df = pd.read_csv('sample_submission.csv')


In [34]:
test_df = test_df.drop(columns=['CreditScore'], errors='ignore')

X_test = preprocess_new_data(
    new_df=test_df,
    input_cols=input_cols,
    encoder=encoder,
    scaler=scaler
)

/content/process_bank_churn_2.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'Gender': {'Female': 0, 'Male': 1}})


In [35]:
# Отримуємо ймовірності для класу 1 (Exited)
test_probs = dt_random_search_best.predict_proba(X_test)[:, 1]

In [49]:
submission_df['Exited'] = (test_probs >= 0.21).astype(int)

In [50]:
submission_df.to_csv('submission_rsb_021.csv', index=False)
submission_df.head()

,id,Exited
0,15000,1
1,15001,0
2,15002,0
3,15003,1
4,15004,0


In [51]:
from google.colab import files
files.download('submission_rsb_021.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

Submission на Kaggle зробила, результат - 0.83157, це покращення від минулого найкращого результату, 27 місце (далеко не продвинулася))